In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain import hub
from langchain.vectorstores import Chroma
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
import pickle
from langchain_community.llms import Ollama
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain import hub
import pathlib
import textwrap
import os
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

genai.configure(api_key=os.environ['GEMINI_API_KEY'])

prompt = hub.pull("mehdixlabetix/rag-law-fr")
llm = Ollama(model="mistral")


# Document loader

In [2]:
import glob

pdf_files = glob.glob("src/*.pdf")
pages = []

for pdf_file in pdf_files:
    pages.extend(PyPDFLoader(pdf_file).load_and_split())

for page in pages:
    page.page_content = page.page_content.replace("Imprimerie Officielle de la République Tunisienne", "")


# Splitter


In [5]:
doc_chunks = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    separators=["\nArticle", "\n\n", "\n", ".", "!", "?", ",", " ", ""],
    chunk_overlap=50,
)
chunks = text_splitter.split_documents(pages)
len(chunks)


28127

# Vectorstores
## for now BAAI/bge-m3 is the best embedder

#### In the cloud (pinecone)

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = "9721efa4-ff98-4bc5-9b28-93919d1657a5"
# embedder = HuggingFaceEmbeddings(
#     model_name = 'dangvantuan/sentence-camembert-large'
# )

index_name = "pfa"

#docsearch = PineconeVectorStore.from_documents(chunks, embedder, index_name=index_name)
#docsearch = PineconeVectorStore(index_name=index_name,embedding=embedder)

#### Locally (chroma)

In [10]:

embedder = HuggingFaceEmbeddings(
    model_name = "BAAI/bge-m3"
)
persist_directory = 'docs/chroma_bge/'
#vectordb.delete_collection()
# vectordb = Chroma.from_documents(
#     documents=chunks,
#     embedding=embedder,
#     persist_directory=persist_directory
# ) 
# vectordb.persist()
vectordb = Chroma(persist_directory="docs/chroma_bge/",embedding_function=embedder)
print(vectordb._collection.count())

28127


# Retrieval
### 4 methodes

In [34]:

query="comment signer un contrat de location  ?"

#needs optimization
# rewrite_prompt = " Fournir une meilleure requête au moteur de recherche web pour répondre à la question donnée,commençant les requêtes par '**' et en terminant les requêtes par '**'.Répondez en une seule phrase. Question :"+query+". Réponse (la réponse doit etre en français):"
# rewrited_prompt=llm.invoke(rewrite_prompt)
# rewrited_prompt = rewrited_prompt.split('**')[1]

#méthode 1 avec retriever
retriever = vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 20})
matched_docs = retriever.get_relevant_documents(query)
# for i, d in enumerate(matched_docs):
#     print(f"\n## Document {i}\n")
#     print(d.page_content)


#méthode 2 avec mmr
found_docs = vectordb.max_marginal_relevance_search(query, k=5, fetch_k=50)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n\n")

#méthode 3 avec similarity search
# sim_docs = vectordb.similarity_search_with_score(rewrited_prompt, k=5)  
# for result in sim_docs:
#     print("\n")
#     print(result[1])
#     print(result[0].page_content)

#méthode 4 avec multiquery qui utilise le llm pour trouver le contexte
# llm_retriever = MultiQueryRetriever.from_llm(retriever=vectordb.as_retriever(), llm=llm)
# unique_docs = llm_retriever.get_relevant_documents(query=query)
# print(unique_docs)
    

1. deux journaux quotidiens dont un en langue arabe, avec l'indication des nantissements et des créanciers inscrits s'il y'en a. 
Le loueur est tenu, dans le même délai, soit de se faire inscrire au 
registre du commerce, soit de faire modifier son inscription personnelle avec la mention expresse de la mise en location. 
La fin de la location donnera lieu aux mêmes mesures de publicité. 


2. 176  CHAPITRE VII 
De la location de coffre-fort 
Article 698.- Le contrat de location de coffre-fort est le contrat par 
lequel une banque met à la disposition du locataire un coffre ou un 
compartiment de coffre, pour une période déterminée, moyennant une redevance. 


3. convention conclue entre les deux parties dans laquelle doivent être mentionnés notamment les lieux d’implantation de ces travaux, leur nature, leur volume, leur montant, les délais de leur exécution, le mode de paiement ainsi que toutes conditions techniques ou autres entrant dans le cadre de l’exécution desdits travaux et ce 

In [ ]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

score = reranker.compute_score(['query', 'passage'])
print(score) # -5.65234375

# You can map the scores into 0-1 by set "normalize=True", which will apply sigmoid function to the score
score = reranker.compute_score(['query', 'passage'], normalize=True)
print(score) # 0.003497010252573502

scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']])
print(scores) # [-8.1875, 5.26171875]

# You can map the scores into 0-1 by set "normalize=True", which will apply sigmoid function to the score
scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']], normalize=True)
print(scores) # [0.00027803096387751553, 0.9948403768236574]



# Prompt Engineering and generation

In [36]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

final_prompt = prompt.invoke({"context":format_docs(found_docs), "question":query})

print(final_prompt)

messages=[HumanMessage(content=" Tu es un assistant juridique spécialisé dans la loi en Tunisie. Ta mission est de répondre aux questions des gens sur différents aspects juridiques ,en te limitant aux informations générales et en évitant les cas sensibles ou extrêmes. Si une question dépasse ton champ d'expertise ou si elle concerne un sujet délicat, tu dois informer l'utilisateur que tu ne peux pas fournir d'aide spécifique dans ce cas. Utilise les pièces suivantes du contexte pour répondre. Utilise un langage simple et accessible pour garantir que tout le monde puisse comprendre tes réponses. Gardez votre réponse brève et efficace maximum quatre ou cinq phrases.\xa0Tu dois répondre en français.\n Question: comment signer un contrat de location  ?.\nContexte: deux journaux quotidiens dont un en langue arabe, avec l'indication des nantissements et des créanciers inscrits s'il y'en a. \nLe loueur est tenu, dans le même délai, soit de se faire inscrire au \nregistre du commerce, soit de 

In [59]:
llm.invoke(final_prompt)

" Pour devenir citoyen tunisien, il faut soumettre une déclaration sur papier timbré en deux exemplaires, avec preuve de domicile et justification de nationalité antérieure. Cette procédure est réglementée par l'article 39 de la loi tunisienne sur l'acquisition de nationalité tunisienne."

In [31]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

model = genai.GenerativeModel('gemini-1.5-pro-latest')

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [43]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


response = model.generate_content(final_prompt.messages[0].content,stream=True)
for chunk in response:
  print(chunk.text)


  

##
 Signature d'un contrat de location en Tunisie:

Pour signer un contrat de
 location en Tunisie, vous devez suivre certaines étapes pour assurer sa validité.  


**1. Éléments essentiels :**  Assurez-vous que le contrat inclut les informations clés comme les noms des parties, la description du bien
, le montant du loyer, la durée de la location et les responsabilités de chaque partie.

**2. Légalisation des signatures :**  
Les signatures du bailleur et du locataire doivent être légalisées par une autorité compétente telle qu'un président de tribunal, un gouverneur, un délégué ou un président de municipalité. 

**3. Enregistrement (fac
ultatif) :**  Bien que non obligatoire, l'enregistrement du contrat au bureau de la conservation de la propriété foncière peut offrir une protection supplémentaire en cas de litige.

**4. Publicité :**  Le ba
illeur doit déclarer la location auprès du registre du commerce ou modifier son inscription personnelle pour mentionner la mise en location du b

In [54]:
response._result.candidates[0].safety_ratings

[category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
]